In [ ]:
# Exploration Notebook for ADAMMA Project
# Konstantinos Kalaitzidis

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Inline plots
%matplotlib inline


In [ ]:
from pathlib import Path

RAW_PATH = Path("../data/raw/ADAMMA.csv")
assert RAW_PATH.exists(), f"File not found: {RAW_PATH}"

df = pd.read_csv(RAW_PATH)
df.head()